In [1]:
from __future__ import print_function

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.backends.cudnn as cudnn

In [3]:
import torchvision
import torchvision.transforms as transforms

In [12]:
import os
import sys
import time
import argparse
import numpy as np

In [5]:
from torch.autograd import Variable

In [28]:
lr = 0.001
dataset = 'mnist'
start_epoch = 1
num_epochs = 10
batch_size = 256
optim_type = 'Adam'

In [10]:
use_cuda = torch.cuda.is_available()

In [15]:
best_acc = 0
resize=32

In [16]:
# Data Uplaod
print('\n[Phase 1] : Data Preparation')

transform_train = transforms.Compose([
    transforms.Resize((resize, resize)),
    transforms.ToTensor(),
]) 

transform_test = transforms.Compose([
    transforms.Resize((resize, resize)),
    transforms.RandomCrop(32, padding=4),
    transforms.ToTensor(),
])


[Phase 1] : Data Preparation


In [17]:
if (dataset == 'mnist'):
    print("| Preparing MNIST dataset...")
    sys.stdout.write("| ")
    trainset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform_train)
    testset = torchvision.datasets.MNIST(root='./data', train=False, download=False, transform=transform_test)
    outputs = 10
    inputs = 1

| Preparing MNIST dataset...
| Downloading http://yann.lecun.com/exdb/mnist/train-images-idx3-ubyte.gz
Processing...
Done!


In [18]:
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=4)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=4)

In [22]:
net = AlexNet(num_classes = 10,inputs=1)
file_name = 'alexnet-'

In [23]:
if use_cuda:
    net.cuda()
    net = torch.nn.DataParallel(net, device_ids=range(torch.cuda.device_count()))
    cudnn.benchmark = True

In [24]:
criterion = nn.CrossEntropyLoss()

In [34]:
def train(epoch):
    net.train()
    train_loss = 0
    correct = 0
    total = 0
    optimizer = optim.Adam(net.parameters(), lr=lr, weight_decay=5e-4)

    print('\n=> Training Epoch #%d, LR=%.4f' %(epoch, lr))
    for batch_idx, (inputs_value, targets) in enumerate(trainloader):
        if use_cuda:
            inputs_value, targets = inputs_value.cuda(), targets.cuda() # GPU settings
        optimizer.zero_grad()
        inputs_value, targets = Variable(inputs_value), Variable(targets)
        outputs = net(inputs_value)               # Forward Propagation
        loss = criterion(outputs, targets)  # Loss
        loss.backward()  # Backward Propagation
        optimizer.step() # Optimizer update

        train_loss += loss.data[0]
        _, predicted = torch.max(outputs.data, 1)
        total += targets.size(0)
        correct += predicted.eq(targets.data).cpu().sum()
        print ('| Epoch [%3d/%3d] Iter[%3d/%3d]\t\tLoss: %.4f Acc@1: %.3f%%'
                %(epoch, num_epochs, batch_idx+1,
                    (len(trainset)//batch_size)+1, loss.data[0], 100.*correct/total))

In [39]:
def test(epoch):
    global best_acc
    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs_value, targets) in enumerate(testloader):
        if use_cuda:
            inputs_value, targets = inputs_value.cuda(), targets.cuda()
        with torch.no_grad():
            inputs_value, targets = Variable(inputs_value), Variable(targets)
        outputs = net(inputs_value)
        loss = criterion(outputs, targets)

        test_loss += loss.data[0]
        _, predicted = torch.max(outputs.data, 1)
        total += targets.size(0)
        correct += predicted.eq(targets.data).cpu().sum()

    # Save checkpoint when best model
    acc = 100.*correct/total
    print("\n| Validation Epoch #%d\t\t\tLoss: %.4f Acc@1: %.2f%%" %(epoch, loss.data[0], acc))
    
    if acc > best_acc:
        best_acc = acc

In [40]:

def conv_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        #nn.init.xavier_uniform(m.weight, gain=np.sqrt(2))
        nn.init.normal_(m.weight, mean=0, std=1)
        nn.init.constant(m.bias, 0)

class AlexNet(nn.Module):

    def __init__(self, num_classes, inputs=3):
        super(AlexNet, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(inputs, 64, kernel_size=11, stride=4, padding=5),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(64, 192, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(192, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )
        self.classifier = nn.Linear(256, num_classes)

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x


In [41]:
def get_hms(seconds):
    m, s = divmod(seconds, 60)
    h, m = divmod(m, 60)

    return h, m, s

In [42]:
elapsed_time = 0
for epoch in range(start_epoch, start_epoch+num_epochs):
    start_time = time.time()

    train(epoch)
    test(epoch)

    epoch_time = time.time() - start_time
    elapsed_time += epoch_time
    print('| Elapsed time : %d:%02d:%02d'  %(get_hms(elapsed_time)))


=> Training Epoch #1, LR=0.0010


/home/insideout/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:19: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
/home/insideout/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:25: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


| Epoch [  1/ 10] Iter[  1/235]		Loss: 0.0232 Acc@1: 99.000%
| Epoch [  1/ 10] Iter[  2/235]		Loss: 0.0340 Acc@1: 99.000%
| Epoch [  1/ 10] Iter[  3/235]		Loss: 0.0068 Acc@1: 99.000%
| Epoch [  1/ 10] Iter[  4/235]		Loss: 0.0366 Acc@1: 99.000%
| Epoch [  1/ 10] Iter[  5/235]		Loss: 0.0028 Acc@1: 99.000%
| Epoch [  1/ 10] Iter[  6/235]		Loss: 0.0185 Acc@1: 99.000%
| Epoch [  1/ 10] Iter[  7/235]		Loss: 0.0209 Acc@1: 99.000%
| Epoch [  1/ 10] Iter[  8/235]		Loss: 0.0400 Acc@1: 99.000%
| Epoch [  1/ 10] Iter[  9/235]		Loss: 0.0157 Acc@1: 99.000%
| Epoch [  1/ 10] Iter[ 10/235]		Loss: 0.0206 Acc@1: 99.000%
| Epoch [  1/ 10] Iter[ 11/235]		Loss: 0.0380 Acc@1: 99.000%
| Epoch [  1/ 10] Iter[ 12/235]		Loss: 0.0336 Acc@1: 99.000%
| Epoch [  1/ 10] Iter[ 13/235]		Loss: 0.0127 Acc@1: 99.000%
| Epoch [  1/ 10] Iter[ 14/235]		Loss: 0.0040 Acc@1: 99.000%
| Epoch [  1/ 10] Iter[ 15/235]		Loss: 0.0223 Acc@1: 99.000%
| Epoch [  1/ 10] Iter[ 16/235]		Loss: 0.0375 Acc@1: 99.000%
| Epoch [  1/ 10] Iter[ 

| Epoch [  1/ 10] Iter[137/235]		Loss: 0.0313 Acc@1: 99.000%
| Epoch [  1/ 10] Iter[138/235]		Loss: 0.0564 Acc@1: 99.000%
| Epoch [  1/ 10] Iter[139/235]		Loss: 0.0783 Acc@1: 99.000%
| Epoch [  1/ 10] Iter[140/235]		Loss: 0.1273 Acc@1: 99.000%
| Epoch [  1/ 10] Iter[141/235]		Loss: 0.0122 Acc@1: 99.000%
| Epoch [  1/ 10] Iter[142/235]		Loss: 0.0263 Acc@1: 99.000%
| Epoch [  1/ 10] Iter[143/235]		Loss: 0.0458 Acc@1: 99.000%
| Epoch [  1/ 10] Iter[144/235]		Loss: 0.0100 Acc@1: 99.000%
| Epoch [  1/ 10] Iter[145/235]		Loss: 0.0331 Acc@1: 99.000%
| Epoch [  1/ 10] Iter[146/235]		Loss: 0.0229 Acc@1: 99.000%
| Epoch [  1/ 10] Iter[147/235]		Loss: 0.0306 Acc@1: 99.000%
| Epoch [  1/ 10] Iter[148/235]		Loss: 0.0108 Acc@1: 99.000%
| Epoch [  1/ 10] Iter[149/235]		Loss: 0.0090 Acc@1: 99.000%
| Epoch [  1/ 10] Iter[150/235]		Loss: 0.0158 Acc@1: 99.000%
| Epoch [  1/ 10] Iter[151/235]		Loss: 0.0128 Acc@1: 99.000%
| Epoch [  1/ 10] Iter[152/235]		Loss: 0.0494 Acc@1: 99.000%
| Epoch [  1/ 10] Iter[1

/home/insideout/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:15: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
  from ipykernel import kernelapp as app


...

| Epoch [  8/ 10] Iter[129/235]		Loss: 0.0385 Acc@1: 99.000%
| Epoch [  8/ 10] Iter[130/235]		Loss: 0.0135 Acc@1: 99.000%
| Epoch [  8/ 10] Iter[131/235]		Loss: 0.0060 Acc@1: 99.000%
| Epoch [  8/ 10] Iter[132/235]		Loss: 0.0071 Acc@1: 99.000%
| Epoch [  8/ 10] Iter[133/235]		Loss: 0.0377 Acc@1: 99.000%
| Epoch [  8/ 10] Iter[134/235]		Loss: 0.0081 Acc@1: 99.000%
| Epoch [  8/ 10] Iter[135/235]		Loss: 0.0196 Acc@1: 99.000%
| Epoch [  8/ 10] Iter[136/235]		Loss: 0.0504 Acc@1: 99.000%
| Epoch [  8/ 10] Iter[137/235]		Loss: 0.0131 Acc@1: 99.000%
| Epoch [  8/ 10] Iter[138/235]		Loss: 0.0033 Acc@1: 99.000%
| Epoch [  8/ 10] Iter[139/235]		Loss: 0.0230 Acc@1: 99.000%
| Epoch [  8/ 10] Iter[140/235]		Loss: 0.0153 Acc@1: 99.000%
| Epoch [  8/ 10] Iter[141/235]		Loss: 0.0407 Acc@1: 99.000%
| Epoch [  8/ 10] Iter[142/235]		Loss: 0.0327 Acc@1: 99.000%
| Epoch [  8/ 10] Iter[143/235]		Loss: 0.0203 Acc@1: 99.000%
| Epoch [  8/ 10] Iter[144/235]		Loss: 0.0060 Acc@1: 99.000%
| Epoch [  8/ 10] Iter[1

| Epoch [  9/ 10] Iter[ 33/235]		Loss: 0.0129 Acc@1: 99.000%
| Epoch [  9/ 10] Iter[ 34/235]		Loss: 0.0026 Acc@1: 99.000%
| Epoch [  9/ 10] Iter[ 35/235]		Loss: 0.0102 Acc@1: 99.000%
| Epoch [  9/ 10] Iter[ 36/235]		Loss: 0.0035 Acc@1: 99.000%
| Epoch [  9/ 10] Iter[ 37/235]		Loss: 0.0080 Acc@1: 99.000%
| Epoch [  9/ 10] Iter[ 38/235]		Loss: 0.0063 Acc@1: 99.000%
| Epoch [  9/ 10] Iter[ 39/235]		Loss: 0.0138 Acc@1: 99.000%
| Epoch [  9/ 10] Iter[ 40/235]		Loss: 0.0019 Acc@1: 99.000%
| Epoch [  9/ 10] Iter[ 41/235]		Loss: 0.0176 Acc@1: 99.000%
| Epoch [  9/ 10] Iter[ 42/235]		Loss: 0.0022 Acc@1: 99.000%
| Epoch [  9/ 10] Iter[ 43/235]		Loss: 0.0040 Acc@1: 99.000%
| Epoch [  9/ 10] Iter[ 44/235]		Loss: 0.0036 Acc@1: 99.000%
| Epoch [  9/ 10] Iter[ 45/235]		Loss: 0.0012 Acc@1: 99.000%
| Epoch [  9/ 10] Iter[ 46/235]		Loss: 0.0189 Acc@1: 99.000%
| Epoch [  9/ 10] Iter[ 47/235]		Loss: 0.0054 Acc@1: 99.000%
| Epoch [  9/ 10] Iter[ 48/235]		Loss: 0.0016 Acc@1: 99.000%
| Epoch [  9/ 10] Iter[ 

| Epoch [  9/ 10] Iter[179/235]		Loss: 0.0113 Acc@1: 99.000%
| Epoch [  9/ 10] Iter[180/235]		Loss: 0.0085 Acc@1: 99.000%
| Epoch [  9/ 10] Iter[181/235]		Loss: 0.0039 Acc@1: 99.000%
| Epoch [  9/ 10] Iter[182/235]		Loss: 0.0075 Acc@1: 99.000%
| Epoch [  9/ 10] Iter[183/235]		Loss: 0.0059 Acc@1: 99.000%
| Epoch [  9/ 10] Iter[184/235]		Loss: 0.0073 Acc@1: 99.000%
| Epoch [  9/ 10] Iter[185/235]		Loss: 0.0146 Acc@1: 99.000%
| Epoch [  9/ 10] Iter[186/235]		Loss: 0.0085 Acc@1: 99.000%
| Epoch [  9/ 10] Iter[187/235]		Loss: 0.0066 Acc@1: 99.000%
| Epoch [  9/ 10] Iter[188/235]		Loss: 0.0064 Acc@1: 99.000%
| Epoch [  9/ 10] Iter[189/235]		Loss: 0.0168 Acc@1: 99.000%
| Epoch [  9/ 10] Iter[190/235]		Loss: 0.0221 Acc@1: 99.000%
| Epoch [  9/ 10] Iter[191/235]		Loss: 0.0075 Acc@1: 99.000%
| Epoch [  9/ 10] Iter[192/235]		Loss: 0.0150 Acc@1: 99.000%
| Epoch [  9/ 10] Iter[193/235]		Loss: 0.0121 Acc@1: 99.000%
| Epoch [  9/ 10] Iter[194/235]		Loss: 0.0075 Acc@1: 99.000%
| Epoch [  9/ 10] Iter[1

| Epoch [ 10/ 10] Iter[ 79/235]		Loss: 0.0124 Acc@1: 99.000%
| Epoch [ 10/ 10] Iter[ 80/235]		Loss: 0.0308 Acc@1: 99.000%
| Epoch [ 10/ 10] Iter[ 81/235]		Loss: 0.0017 Acc@1: 99.000%
| Epoch [ 10/ 10] Iter[ 82/235]		Loss: 0.0036 Acc@1: 99.000%
| Epoch [ 10/ 10] Iter[ 83/235]		Loss: 0.0089 Acc@1: 99.000%
| Epoch [ 10/ 10] Iter[ 84/235]		Loss: 0.0136 Acc@1: 99.000%
| Epoch [ 10/ 10] Iter[ 85/235]		Loss: 0.0077 Acc@1: 99.000%
| Epoch [ 10/ 10] Iter[ 86/235]		Loss: 0.0099 Acc@1: 99.000%
| Epoch [ 10/ 10] Iter[ 87/235]		Loss: 0.0013 Acc@1: 99.000%
| Epoch [ 10/ 10] Iter[ 88/235]		Loss: 0.0486 Acc@1: 99.000%
| Epoch [ 10/ 10] Iter[ 89/235]		Loss: 0.0374 Acc@1: 99.000%
| Epoch [ 10/ 10] Iter[ 90/235]		Loss: 0.0045 Acc@1: 99.000%
| Epoch [ 10/ 10] Iter[ 91/235]		Loss: 0.0101 Acc@1: 99.000%
| Epoch [ 10/ 10] Iter[ 92/235]		Loss: 0.0018 Acc@1: 99.000%
| Epoch [ 10/ 10] Iter[ 93/235]		Loss: 0.0149 Acc@1: 99.000%
| Epoch [ 10/ 10] Iter[ 94/235]		Loss: 0.0088 Acc@1: 99.000%
| Epoch [ 10/ 10] Iter[ 

| Epoch [ 10/ 10] Iter[217/235]		Loss: 0.0054 Acc@1: 99.000%
| Epoch [ 10/ 10] Iter[218/235]		Loss: 0.0330 Acc@1: 99.000%
| Epoch [ 10/ 10] Iter[219/235]		Loss: 0.0087 Acc@1: 99.000%
| Epoch [ 10/ 10] Iter[220/235]		Loss: 0.0153 Acc@1: 99.000%
| Epoch [ 10/ 10] Iter[221/235]		Loss: 0.0184 Acc@1: 99.000%
| Epoch [ 10/ 10] Iter[222/235]		Loss: 0.0083 Acc@1: 99.000%
| Epoch [ 10/ 10] Iter[223/235]		Loss: 0.0037 Acc@1: 99.000%
| Epoch [ 10/ 10] Iter[224/235]		Loss: 0.0200 Acc@1: 99.000%
| Epoch [ 10/ 10] Iter[225/235]		Loss: 0.0151 Acc@1: 99.000%
| Epoch [ 10/ 10] Iter[226/235]		Loss: 0.0115 Acc@1: 99.000%
| Epoch [ 10/ 10] Iter[227/235]		Loss: 0.0129 Acc@1: 99.000%
| Epoch [ 10/ 10] Iter[228/235]		Loss: 0.0060 Acc@1: 99.000%
| Epoch [ 10/ 10] Iter[229/235]		Loss: 0.0034 Acc@1: 99.000%
| Epoch [ 10/ 10] Iter[230/235]		Loss: 0.0455 Acc@1: 99.000%
| Epoch [ 10/ 10] Iter[231/235]		Loss: 0.0303 Acc@1: 99.000%
| Epoch [ 10/ 10] Iter[232/235]		Loss: 0.0120 Acc@1: 99.000%
| Epoch [ 10/ 10] Iter[2

In [43]:
print('\n[Phase 4] : Testing model')
print('* Test results : Acc@1 = %.2f%%' %(best_acc))


[Phase 4] : Testing model
* Test results : Acc@1 = 85.00%
